# Lecture 19
## Monday, November 13th 2017
### Joins with `SQLite`, `pandas`

## Starting Up
You can connect to the saved database from last time if you want.  Alternatively, for extra practice, you can just recreate it from the datasets provided in the `.txt` files.  That's what I'll do.

In [1]:
import sqlite3
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('L19DB_demo.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    return pd.DataFrame.from_items(framelist)

## Recap
Last time, you played with a bunch of `SQLite` commands to query and update the tables in the database.

One thing we didn't get to was how to query the contributors table based off of a query in the candidates table.  For example, suppose you want to query which contributors donated to Obama. You could use a nested `SELECT` statement to accomplish that.

In [2]:
query = '''SELECT * FROM contributors WHERE candidate_id = (SELECT id from candidates WHERE last_name = "Obama")'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,26,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,50.00,2007-07-30,20
1,27,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,25.00,2007-08-16,20
2,28,Buckheit,Bruce,,8904 KAREN DR,,FAIRFAX,VA,220312731,100.00,2007-09-19,20
3,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.00,2007-08-14,20
4,30,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,-2300.00,2007-08-14,20
5,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.00,2007-08-14,20
6,32,Buck,Thomas,,4206 Terrace Street,,Kansas City,MO,64111,100.00,2007-09-25,20
7,33,Buck,Jay,K.,1855 Old Willow Rd Unit 322,,Northfield,IL,600932918,200.00,2007-09-12,20
8,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.00,2007-09-30,20
9,35,Buck,Barbara,,1780 NE 138th St,,North Miami,FL,331811316,50.00,2007-09-13,20


# Joins
The last example involved querying data from multiple tables.

In particular, we combined columns from the two related tables (related through the `FOREIGN KEY`).

This leads to the idea of *joining* multiple tables together.  `SQL` has a set of commands to handle different types of joins.  `SQLite` does not support the full suite of join commands offered by `SQL` but you should still be able to get the main ideas from the limited command set.

We'll begin with the `INNER JOIN`.

## INNER JOIN
The idea here is that you will combine the tables if the values of certain columns are the same between the two tables.  In our example, we will join the two tables based on the candidate id.  The result of the `INNER JOIN` will be a new table consisting of the columns we requested and containing the common data.  Since we are joining based off of the candidate id, we will not be excluding any rows.

### Example

Here are two tables.  Table A has the form:

| nA   | attr  | idA |
| :::: | ::::: | ::: |
| s1   | 23    | 0   |
| s2   | 7     | 2   |

and table B has the form:

| nB   | attr  | idB |
| :::: | ::::: | ::: |
| t1   | 60    | 0   |
| t2   | 14    | 7   |
| t3   | 22    | 2   |

Table A is associated with Table B through a foreign key on the id column.

If we join the two tables by comparing the id columns and selecting the n1, n2, and attr columns then we'll get 

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     |
| s2   | 7       | t3  | 22     |


The `SQLite` code to do this join would be 

```sql
SELECT nA, A.attr, nB, B.attr FROM A INNER JOIN B ON B.idB = A.idA
```

Notice that the second row in table B is gone because the id values are not the same.

### Thoughts
What is `SQL` doing with this operation?  It may help to visualize this with a Venn diagram.  Table A has rows with values corresponding to the `idA` attribute.  Column B has rows with values corresponding to the `idB` attribute.  The `INNER JOIN` will combine the two tables such that rows with common entries in the `id` attributes are included.  We essentially have the following Venn diagram.

![](inner_join.png)

## Exercises
1. Using an `INNER JOIN`, join the candidates and contributors tables by comparing the `candidate_id` and `candidates_id` columns.  Display your joined table with columns `contributors.last_name`, `contributors.first_name`, and `candidates.last_name`.
2. Do the same inner join as in the last part, but this time append a `WHERE` clause to select a specific candidate's last name.